In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import os
import time
import requests
import warnings
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_excel('Resources/Draft_Players.xlsx')
df

Round  Pick Team                Player        First       Last Position  \
0        1     1  HOU        Mario Williams        Mario   Williams       DE   
1        1     2  NOR           Reggie Bush       Reggie       Bush       RB   
2        1     3  TEN           Vince Young        Vince      Young       QB   
3        1     4  NYJ  DBrickashaw Ferguson  DBrickashaw   Ferguson        T   
4        1     5  GNB               AJ Hawk           AJ       Hawk       LB   
..     ...   ...  ...                   ...          ...        ...      ...   
250      7   251  HOU        David Anderson        David   Anderson       WR   
251      7   252  NOR       Marques Colston      Marques    Colston       WR   
252      7   253  GNB        Dave Tollefson         Dave  Tollefson       DE   
253      7   254  SFO        Vickiel Vaughn      Vickiel     Vaughn       DB   
254      7   255  OAK         Kevin McMahan        Kevin    McMahan       WR   

      Age   Until  AP1  ...  RushAtt  RushYards  RushTDs  Rec  RecYards  \
0    21.0  2016.0    1  ...        0          0        0    0         0   
1    21.0  2016.0    0  ...     1286       5490       36  477      3598   
2    23.0  2011.0    0  ...      282       1459       12    0         0   
3    22.0  2015.0    0  ...        0          0        0    0         0   
4    22.0  2016.0    0  ...        0          0        0    0         0   
..    ...     ...  ...  ...      ...        ...      ...  ...       ...   
250  23.0  2011.0    0  ...        0          0        0   88       965   
251  23.0  2015.0    0  ...        2          7        0  711      9759   
252  24.0  2012.0    0  ...        0          0        0    0         0   
253   NaN     NaN    0  ...        0          0        0    0         0   
254   NaN     NaN    0  ...        0          0        0    0         0   

     RecTDs  TackleSolo  DefensiveInt  Sacks             College  
0         0         317             0   97.5  North Carolina St.  
1        18           0             0    0.0                 USC  
2         0           0             0    0.0               Texas  
3         0           0             0    0.0            Virginia  
4         0         645             9   20.0            Ohio St.  
..      ...         ...           ...    ...                 ...  
250       4           8             0    0.0        Colorado St.  
251      72           1             0    0.0             Hofstra  
252       0          62             0   10.5     NW Missouri St.  
253       0           0             0    0.0            Arkansas  
254       0           0             0    0.0               Maine  

[255 rows x 30 columns]

In [3]:
players = df['Player'].tolist()
players

['Mario Williams',
 'Reggie Bush',
 'Vince Young',
 'DBrickashaw Ferguson',
 'AJ Hawk',
 'Vernon Davis',
 'Michael Huff',
 'Donte Whitner',
 'Ernie Sims',
 'Matt Leinart',
 'Jay Cutler',
 'Haloti Ngata',
 'Kamerion Wimbley',
 'Brodrick Bunkley',
 'Tye Hill',
 'Jason Allen',
 'Chad Greenway',
 'Bobby Carpenter',
 'Antonio Cromartie',
 'Tamba Hali',
 'Laurence Maroney',
 'Manny Lawson',
 'Davin Joseph',
 'Johnathan Joseph',
 'Santonio Holmes',
 'John McCargo',
 'DeAngelo Williams',
 'Marcedes Lewis',
 'Nick Mangold',
 'Joseph Addai',
 'Kelly Jennings',
 'Mathias Kiwanuka',
 'DeMeco Ryans',
 'DQwell Jackson',
 'Rocky McIntosh',
 'Chad Jackson',
 'Jimmy Williams',
 'Thomas Howard',
 'Winston Justice',
 'Daniel Bullocks',
 'Deuce Lutui',
 'Danieal Manning',
 'Roman Harper',
 'Sinorice Moss',
 'LenDale White',
 'Joe Klopfenstein',
 'Daryn Colledge',
 'Cedric Griffin',
 'Kellen Clemens',
 'Marcus McNeill',
 'Ryan Cook',
 'Greg Jennings',
 'Anthony Fasano',
 'Bernard Pollard',
 'Andrew Whitwor

In [4]:
qb_data = pd.DataFrame(columns = ['Player', 'passCmp', 'passAtt', 'passPct', 'passYds', 'passY/A', 'passAY/A', 'passTD', 'passInt', 'passRate'])
rb_data = pd.DataFrame(columns = ['Player', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'recRec', 'recYds', 'recAvg', 'recTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD'])
wr_te_data = pd.DataFrame(columns = ['Player', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD'])
k_data = pd.DataFrame(columns = ['Player', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts', 'Punts', 'puntYds', 'puntAvg'])
p_data = pd.DataFrame(columns = ['Player', 'Punts', 'puntYds', 'puntAvg', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts'])
def_data = pd.DataFrame(columns = ['Player', 'tackleSolo', 'tackleAst', 'tackleTot', 'tackleLoss', 'Sk', 'defInt', 'defYds', 'defAvg', 'defIntTD', 'defIntPD', 'fumFR', 'fumYds', 'fumTD', 'fumFF'])

In [5]:
x = 0

for _ in df:
    try:
        pre_url = "https://www.sports-reference.com/cfb/players/"
        post_url = "-1.html"
        first = df.First[x].lower()
        last = df.Last[x].lower()
        full_url = f'{pre_url}{first}-{last}{post_url}'
    
        if df.Position[x] == 'QB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'passCmp', 'passAtt', 'passPct', 'passYds', 'passY/A', 'passAY/A', 'passTD', 'passInt', 'passRate']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            qb_data = pd.merge_ordered(qb_data, df_player)
        
        elif df.Position[x] == 'RB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'recRec', 'recYds', 'recAvg', 'recTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            rb_data = pd.merge_ordered(rb_data, df_player)
            
        elif df.Position[x] == 'WR':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'TE':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'K':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts', 'Punts', 'puntYds', 'puntAvg']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            k_data = pd.merge_ordered(k_data, df_player)
        
        elif df.Position[x] == 'P':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'Punts', 'puntYds', 'puntAvg', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            p_data = pd.merge_ordered(p_data, df_player)
    
        else:
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'tackleSolo', 'tackleAst', 'tackleTot', 'tackleLoss', 'Sk', 'defInt', 'defYds', 'defAvg', 'defIntTD', 'defIntPD', 'fumFR', 'fumYds', 'fumTD', 'fumFF']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            def_data = pd.merge_ordered(def_data, df_player)
        
    except:
        pass
    x = x + 1
    print(x)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [6]:
x = 30

for _ in df:
    try:
        pre_url = "https://www.sports-reference.com/cfb/players/"
        post_url = "-1.html"
        first = df.First[x].lower()
        last = df.Last[x].lower()
        full_url = f'{pre_url}{first}-{last}{post_url}'
    
        if df.Position[x] == 'QB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'passCmp', 'passAtt', 'passPct', 'passYds', 'passY/A', 'passAY/A', 'passTD', 'passInt', 'passRate']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            qb_data = pd.merge_ordered(qb_data, df_player)
        
        elif df.Position[x] == 'RB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'recRec', 'recYds', 'recAvg', 'recTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            rb_data = pd.merge_ordered(rb_data, df_player)
            
        elif df.Position[x] == 'WR':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'TE':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'K':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts', 'Punts', 'puntYds', 'puntAvg']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            k_data = pd.merge_ordered(k_data, df_player)
        
        elif df.Position[x] == 'P':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'Punts', 'puntYds', 'puntAvg', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            p_data = pd.merge_ordered(p_data, df_player)
    
        else:
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'tackleSolo', 'tackleAst', 'tackleTot', 'tackleLoss', 'Sk', 'defInt', 'defYds', 'defAvg', 'defIntTD', 'defIntPD', 'fumFR', 'fumYds', 'fumTD', 'fumFF']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            def_data = pd.merge_ordered(def_data, df_player)
        
    except:
        pass
    x = x + 1
    print(x)

31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [7]:
x = 60

for _ in df:
    try:
        pre_url = "https://www.sports-reference.com/cfb/players/"
        post_url = "-1.html"
        first = df.First[x].lower()
        last = df.Last[x].lower()
        full_url = f'{pre_url}{first}-{last}{post_url}'
    
        if df.Position[x] == 'QB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'passCmp', 'passAtt', 'passPct', 'passYds', 'passY/A', 'passAY/A', 'passTD', 'passInt', 'passRate']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            qb_data = pd.merge_ordered(qb_data, df_player)
        
        elif df.Position[x] == 'RB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'recRec', 'recYds', 'recAvg', 'recTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            rb_data = pd.merge_ordered(rb_data, df_player)
            
        elif df.Position[x] == 'WR':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'TE':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'K':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts', 'Punts', 'puntYds', 'puntAvg']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            k_data = pd.merge_ordered(k_data, df_player)
        
        elif df.Position[x] == 'P':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'Punts', 'puntYds', 'puntAvg', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            p_data = pd.merge_ordered(p_data, df_player)
    
        else:
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'tackleSolo', 'tackleAst', 'tackleTot', 'tackleLoss', 'Sk', 'defInt', 'defYds', 'defAvg', 'defIntTD', 'defIntPD', 'fumFR', 'fumYds', 'fumTD', 'fumFF']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            def_data = pd.merge_ordered(def_data, df_player)
        
    except:
        pass
    x = x + 1
    print(x)

61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90


In [8]:
x = 90

for _ in df:
    try:
        pre_url = "https://www.sports-reference.com/cfb/players/"
        post_url = "-1.html"
        first = df.First[x].lower()
        last = df.Last[x].lower()
        full_url = f'{pre_url}{first}-{last}{post_url}'
    
        if df.Position[x] == 'QB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'passCmp', 'passAtt', 'passPct', 'passYds', 'passY/A', 'passAY/A', 'passTD', 'passInt', 'passRate']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            qb_data = pd.merge_ordered(qb_data, df_player)
        
        elif df.Position[x] == 'RB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'recRec', 'recYds', 'recAvg', 'recTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            rb_data = pd.merge_ordered(rb_data, df_player)
            
        elif df.Position[x] == 'WR':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'TE':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'K':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts', 'Punts', 'puntYds', 'puntAvg']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            k_data = pd.merge_ordered(k_data, df_player)
        
        elif df.Position[x] == 'P':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'Punts', 'puntYds', 'puntAvg', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            p_data = pd.merge_ordered(p_data, df_player)
    
        else:
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'tackleSolo', 'tackleAst', 'tackleTot', 'tackleLoss', 'Sk', 'defInt', 'defYds', 'defAvg', 'defIntTD', 'defIntPD', 'fumFR', 'fumYds', 'fumTD', 'fumFF']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            def_data = pd.merge_ordered(def_data, df_player)
        
    except:
        pass
    x = x + 1
    print(x)

91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


In [9]:
x = 120

for _ in df:
    try:
        pre_url = "https://www.sports-reference.com/cfb/players/"
        post_url = "-1.html"
        first = df.First[x].lower()
        last = df.Last[x].lower()
        full_url = f'{pre_url}{first}-{last}{post_url}'
    
        if df.Position[x] == 'QB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'passCmp', 'passAtt', 'passPct', 'passYds', 'passY/A', 'passAY/A', 'passTD', 'passInt', 'passRate']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            qb_data = pd.merge_ordered(qb_data, df_player)
        
        elif df.Position[x] == 'RB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'recRec', 'recYds', 'recAvg', 'recTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            rb_data = pd.merge_ordered(rb_data, df_player)
            
        elif df.Position[x] == 'WR':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'TE':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'K':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts', 'Punts', 'puntYds', 'puntAvg']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            k_data = pd.merge_ordered(k_data, df_player)
        
        elif df.Position[x] == 'P':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'Punts', 'puntYds', 'puntAvg', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            p_data = pd.merge_ordered(p_data, df_player)
    
        else:
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'tackleSolo', 'tackleAst', 'tackleTot', 'tackleLoss', 'Sk', 'defInt', 'defYds', 'defAvg', 'defIntTD', 'defIntPD', 'fumFR', 'fumYds', 'fumTD', 'fumFF']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            def_data = pd.merge_ordered(def_data, df_player)
        
    except:
        pass
    x = x + 1
    print(x)

121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150


In [10]:
x = 150

for _ in df:
    try:
        pre_url = "https://www.sports-reference.com/cfb/players/"
        post_url = "-1.html"
        first = df.First[x].lower()
        last = df.Last[x].lower()
        full_url = f'{pre_url}{first}-{last}{post_url}'
    
        if df.Position[x] == 'QB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'passCmp', 'passAtt', 'passPct', 'passYds', 'passY/A', 'passAY/A', 'passTD', 'passInt', 'passRate']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            qb_data = pd.merge_ordered(qb_data, df_player)
        
        elif df.Position[x] == 'RB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'recRec', 'recYds', 'recAvg', 'recTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            rb_data = pd.merge_ordered(rb_data, df_player)
            
        elif df.Position[x] == 'WR':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'TE':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'K':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts', 'Punts', 'puntYds', 'puntAvg']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            k_data = pd.merge_ordered(k_data, df_player)
        
        elif df.Position[x] == 'P':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'Punts', 'puntYds', 'puntAvg', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            p_data = pd.merge_ordered(p_data, df_player)
    
        else:
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'tackleSolo', 'tackleAst', 'tackleTot', 'tackleLoss', 'Sk', 'defInt', 'defYds', 'defAvg', 'defIntTD', 'defIntPD', 'fumFR', 'fumYds', 'fumTD', 'fumFF']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            def_data = pd.merge_ordered(def_data, df_player)
        
    except:
        pass
    x = x + 1
    print(x)

151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180


In [11]:
x = 180

for _ in df:
    try:
        pre_url = "https://www.sports-reference.com/cfb/players/"
        post_url = "-1.html"
        first = df.First[x].lower()
        last = df.Last[x].lower()
        full_url = f'{pre_url}{first}-{last}{post_url}'
    
        if df.Position[x] == 'QB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'passCmp', 'passAtt', 'passPct', 'passYds', 'passY/A', 'passAY/A', 'passTD', 'passInt', 'passRate']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            qb_data = pd.merge_ordered(qb_data, df_player)
        
        elif df.Position[x] == 'RB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'recRec', 'recYds', 'recAvg', 'recTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            rb_data = pd.merge_ordered(rb_data, df_player)
            
        elif df.Position[x] == 'WR':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'TE':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'K':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts', 'Punts', 'puntYds', 'puntAvg']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            k_data = pd.merge_ordered(k_data, df_player)
        
        elif df.Position[x] == 'P':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'Punts', 'puntYds', 'puntAvg', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            p_data = pd.merge_ordered(p_data, df_player)
    
        else:
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'tackleSolo', 'tackleAst', 'tackleTot', 'tackleLoss', 'Sk', 'defInt', 'defYds', 'defAvg', 'defIntTD', 'defIntPD', 'fumFR', 'fumYds', 'fumTD', 'fumFF']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            def_data = pd.merge_ordered(def_data, df_player)
        
    except:
        pass
    x = x + 1
    print(x)

181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210


In [12]:
x = 210

for _ in df:
    try:
        pre_url = "https://www.sports-reference.com/cfb/players/"
        post_url = "-1.html"
        first = df.First[x].lower()
        last = df.Last[x].lower()
        full_url = f'{pre_url}{first}-{last}{post_url}'
    
        if df.Position[x] == 'QB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'passCmp', 'passAtt', 'passPct', 'passYds', 'passY/A', 'passAY/A', 'passTD', 'passInt', 'passRate']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            qb_data = pd.merge_ordered(qb_data, df_player)
        
        elif df.Position[x] == 'RB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'recRec', 'recYds', 'recAvg', 'recTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            rb_data = pd.merge_ordered(rb_data, df_player)
            
        elif df.Position[x] == 'WR':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'TE':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'K':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts', 'Punts', 'puntYds', 'puntAvg']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            k_data = pd.merge_ordered(k_data, df_player)
        
        elif df.Position[x] == 'P':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'Punts', 'puntYds', 'puntAvg', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            p_data = pd.merge_ordered(p_data, df_player)
    
        else:
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'tackleSolo', 'tackleAst', 'tackleTot', 'tackleLoss', 'Sk', 'defInt', 'defYds', 'defAvg', 'defIntTD', 'defIntPD', 'fumFR', 'fumYds', 'fumTD', 'fumFF']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            def_data = pd.merge_ordered(def_data, df_player)
        
    except:
        pass
    x = x + 1
    print(x)

211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240


In [13]:
x = 240

for _ in df:
    try:
        pre_url = "https://www.sports-reference.com/cfb/players/"
        post_url = "-1.html"
        first = df.First[x].lower()
        last = df.Last[x].lower()
        full_url = f'{pre_url}{first}-{last}{post_url}'
    
        if df.Position[x] == 'QB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'passCmp', 'passAtt', 'passPct', 'passYds', 'passY/A', 'passAY/A', 'passTD', 'passInt', 'passRate']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            qb_data = pd.merge_ordered(qb_data, df_player)
        
        elif df.Position[x] == 'RB':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'recRec', 'recYds', 'recAvg', 'recTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            rb_data = pd.merge_ordered(rb_data, df_player)
            
        elif df.Position[x] == 'WR':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'TE':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'recRec', 'recYds', 'recAvg', 'recTD', 'rushAtt', 'rushYds', 'rushAvg', 'rushTD', 'scrPlays', 'scrYds', 'scrAvg', 'scrTD']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            wr_te_data = pd.merge_ordered(wr_te_data, df_player)
        
        elif df.Position[x] == 'K':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts', 'Punts', 'puntYds', 'puntAvg']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            k_data = pd.merge_ordered(k_data, df_player)
        
        elif df.Position[x] == 'P':
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'Punts', 'puntYds', 'puntAvg', 'XPM', 'XPA', 'XP%', 'FGM', 'FGA', 'FG%', 'Pts']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            p_data = pd.merge_ordered(p_data, df_player)
    
        else:
            df_player = pd.read_html(full_url, skiprows=0)[0]
            df_player.columns = ['Player', 'School', 'Conf', 'Class', 'Pos', 'G', 'tackleSolo', 'tackleAst', 'tackleTot', 'tackleLoss', 'Sk', 'defInt', 'defYds', 'defAvg', 'defIntTD', 'defIntPD', 'fumFR', 'fumYds', 'fumTD', 'fumFF']
            df_player = df_player[df_player.Player == 'Career']
            df_player = df_player.reset_index(drop=True)
            df_player.iloc[0,0] = df.Player[x]
            df_player.drop(df_player.columns[[1,2,3,4,5]], axis=1, inplace = True)
            def_data = pd.merge_ordered(def_data, df_player)
        
    except:
        pass
    x = x + 1
    print(x)

241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270


In [14]:
def_data


Player  tackleSolo  tackleAst  tackleTot  tackleLoss   Sk  \
0            AJ Hawk        69.0       52.0      121.0        16.5  9.5   
1       AJ Nicholson        53.0       47.0      100.0         9.5  1.0   
2         Aaron Merz         1.0        0.0        1.0         0.0  0.0   
3        Abdul Hodge        96.0       62.0      158.0        11.0  1.0   
4      Alan Zemaitis        37.0       14.0       51.0         2.5  0.0   
..               ...         ...        ...        ...         ...  ...   
104         Tye Hill        35.0        8.0       43.0         4.0  0.0   
105    Tyrone Culver        23.0       21.0       44.0         2.0  2.0   
106   Vickiel Vaughn        46.0       20.0       66.0         2.0  0.0   
107  Victor Adeyanju        32.0       15.0       47.0        12.0  6.5   
108      Zach Strief         0.0        1.0        1.0         0.0  0.0   

     defInt  defYds  defAvg  defIntTD  defIntPD  fumFR  fumYds  fumTD  fumFF  
0       7.0   123.0    17.6       1.0       NaN    NaN     NaN    NaN    NaN  
1       3.0    59.0    19.7       1.0       NaN    1.0    23.0    1.0    NaN  
2       NaN     NaN     NaN       NaN       0.0    0.0     0.0    0.0    0.0  
3       NaN     NaN     NaN       NaN       0.0    1.0    47.0    0.0    3.0  
4      12.0   269.0    22.4       1.0       NaN    NaN    51.0    2.0    NaN  
..      ...     ...     ...       ...       ...    ...     ...    ...    ...  
104     5.0    33.0     6.6       0.0       NaN    NaN     NaN    NaN    NaN  
105     5.0    72.0    14.4       0.0       NaN    NaN    30.0    1.0    NaN  
106     7.0   105.0    15.0       1.0       NaN    NaN     NaN    NaN    NaN  
107     NaN     NaN     NaN       NaN       2.0    1.0     4.0    1.0    1.0  
108     NaN     NaN     NaN       NaN       0.0    0.0     0.0    0.0    0.0  

[109 rows x 15 columns]

In [15]:
rb_data


Player  rushAtt  rushYds  rushAvg  rushTD  recRec  recYds  \
0       Brian Calhoun      619     2760      4.5      27      90     909   
1        Cedric Humes      373     1805      4.8      21      29     224   
2       David Kirtman       46      470     10.2       2      21      94   
3   DeAngelo Williams      969     6026      6.2      55      70     723   
4          JD Runnels       51      450      8.8       5       2       5   
5     Jerious Norwood      573     3222      5.6      15      43     186   
6     Jerome Harrison      482     2800      5.8      25      34     275   
7        Joseph Addai      490     2576      5.3      18      66     641   
8     Justin Hamilton       75      336      4.5       3      34     369   
9    Laurence Maroney      660     3933      6.0      32      21     197   
10      LenDale White      541     3159      5.8      52      31     331   
11    Leon Washington      369     2041      5.5      12      55     481   
12         PJ Daniels      707     3346      4.7      23      56     369   
13    Quinton Ganther      314     1774      5.6       9      32     363   
14        Reggie Bush      433     3169      7.3      25      95    1301   
15         Wali Lundy      742     3193      4.3      43     114     895   

    recAvg  recTD  scrPlays  scrYds  scrAvg  scrTD  
0     10.1      4       709    3669     5.2     31  
1      7.7      0       402    2029     5.0     21  
2      4.5      1        67     564     8.4      3  
3     10.3      5      1039    6749     6.5     60  
4      2.5      0        53     455     8.6      5  
5      4.3      2       616    3408     5.5     17  
6      8.1      1       516    3075     6.0     26  
7      9.7      6       556    3217     5.8     24  
8     10.9      1       109     705     6.5      4  
9      9.4      1       681    4130     6.1     33  
10    10.7      5       572    3490     6.1     57  
11     8.7      1       424    2522     5.9     13  
12     6.6      3       763    3715     4.9     26  
13    11.3      2       346    2137     6.2     11  
14    13.7     13       528    4470     8.5     38  
15     7.9      9       856    4088     4.8     52

In [16]:
k_data


Player  XPM  XPA   XP%  FGM  FGA   FG%  Pts  Punts  puntYds  \
0          Kurt Smith   22   24  91.7    7   12  58.3   43    0.0      0.0   
1  Stephen Gostkowski  159  165  96.4   70   92  76.1  369    NaN      NaN   

   puntAvg  
0      NaN  
1      NaN

In [17]:
p_data


Player  Punts  puntYds  puntAvg   XPM   XPA    XP%  FGM   FGA  \
0  Ryan Plackemeier    220     9957     45.3  31.0  31.0  100.0  9.0  18.0   
1          Sam Koch    134     5902     44.0   NaN   NaN    NaN  NaN   NaN   

    FG%   Pts  
0  50.0  58.0  
1   NaN   NaN

In [18]:
wr_te_data


Player  recRec  recYds  recAvg  recTD  rushAtt  rushYds  \
0       Anthony Fasano      92    1112    12.1      8      NaN      NaN   
1          Ben Obomanu      97    1313    13.5     18      8.0    154.0   
2       Bennie Brazell      17     340    20.0      3      NaN      NaN   
3     Brandon Marshall     112    1674    14.9     13      3.0      5.0   
4         Chad Jackson     120    1586    13.2     16     17.0     90.0   
5        Charles Davis      78     865    11.1      4      NaN      NaN   
6         Cory Rodgers     150    2111    14.1     17     46.0    115.0   
7       David Anderson     223    3634    16.3     20      4.0     18.0   
8         David Thomas      98    1367    13.9     15      NaN      NaN   
9   Demetrius Williams     162    2660    16.4     20     12.0     76.0   
10         Derek Hagan     258    3939    15.3     27      NaN      NaN   
11    Devin Aromashodu      71    1333    18.8      9      1.0      6.0   
12       Domenik Hixon     141    2092    14.8     14      1.0     24.0   
13      Dominique Byrd      81     968    12.0      4      1.0      1.0   
14       Greg Jennings     238    3539    14.9     39     18.0     92.0   
15         Jason Avant     169    2247    13.3     13      NaN      NaN   
16       Jason Pociask      12     144    12.0      0      NaN      NaN   
17           Jeff King      58     724    12.5     12      NaN      NaN   
18           Jeff Webb      40     672    16.8      1      6.0     42.0   
19    Joe Klopfenstein      86    1076    12.5     13      NaN      NaN   
20        Jonathan Orr     107    1824    17.0     19      NaN      NaN   
21        Leonard Pope      65    1044    16.1     10      NaN      NaN   
22      Marcedes Lewis     126    1571    12.5     21      NaN      NaN   
23     Maurice Stovall     130    2195    16.9     18      2.0      1.0   
24           Mike Hass     220    3924    17.8     20      2.0     18.0   
25        Owen Daniels      62     852    13.7      8      4.0     18.0   
26    Quinn Sypniewski      44     563    12.8      5      NaN      NaN   
27     Santonio Holmes     140    2295    16.4     25      3.0     18.0   
28       Sinorice Moss      68    1106    16.3      9      9.0     59.0   
29        Skyler Green     111    1129    10.2      9     31.0    258.0   
30         TJ Williams      26     297    11.4      3      1.0      7.0   
31       Tim Massaquoi      46     489    10.6      2      NaN      NaN   
32        Todd Watkins     101    1720    17.0     15      1.0     -4.0   
33      Tony Scheffler     117    1345    11.5     13      NaN      NaN   
34       Travis Wilson      30     163     5.4      1      7.0     95.0   
35        Vernon Davis      83    1371    16.5      9      2.0      5.0   
36         Willie Reid      91    1046    11.5      3     63.0    309.0   

    rushAvg  rushTD  scrPlays  scrYds  scrAvg  scrTD  
0       NaN     NaN        92    1112    12.1      8  
1      19.3     2.0       105    1467    14.0     20  
2       NaN     NaN        17     340    20.0      3  
3       1.7     0.0       115    1679    14.6     13  
4       5.3     2.0       137    1676    12.2     18  
5       NaN     NaN        78     865    11.1      4  
6       2.5    10.0       196    2226    11.4     27  
7       4.5     0.0       227    3652    16.1     20  
8       NaN     NaN        98    1367    13.9     15  
9       6.3     1.0       174    2736    15.7     21  
10      NaN     NaN       258    3939    15.3     27  
11      6.0     0.0        72    1339    18.6      9  
12     24.0     0.0       142    2116    14.9     14  
13      1.0     0.0        82     969    11.8      4  
14      5.1     0.0       256    3631    14.2     39  
15      NaN     NaN       169    2247    13.3     13  
16      NaN     NaN        12     144    12.0      0  
17      NaN     NaN        58     724    12.5     12  
18      7.0     0.0        46     714    15.5      1  
19      NaN     NaN        86    1076    12.5     13  
20      Na

In [19]:
qb_data

Player  passCmp  passAtt  passPct  passYds  passY/A  passAY/A  \
0           Brad Smith      818     1454     56.3     8644      5.9       5.7   
1        Brodie Croyle      488      869     56.2     6382      7.3       7.1   
2     Bruce Gradkowski      766     1123     68.2     9225      8.2       8.6   
3   Charlie Whitehurst      817     1368     59.7     9665      7.1       6.3   
4          DJ Shockley      240      443     54.2     3555      8.0       8.6   
5         Ingle Martin       54       87     62.1      750      8.6       8.3   
6           Jay Cutler      710     1242     57.2     8697      7.0       6.6   
7       Kellen Clemens      613     1005     61.0     7555      7.5       7.7   
8         Matt Leinart      807     1245     64.8    10693      8.6       9.3   
9          Omar Jacobs      523      811     64.5     6938      8.6       9.7   
10       Reggie McNeal      478      875     54.6     6992      8.0       7.8   
11    Tarvaris Jackson       17       48     35.4      196      4.1       1.7   
12         Vince Young      444      718     61.8     6040      8.4       7.9   

    passTD  passInt  passRate  
0       56       32     114.5  
1       41       22     128.4  
2       85       27     157.4  
3       49       46     124.2  
4       34        9     142.8  
5        3        2     141.3  
6       59       36     125.9  
7       61       24     139.4  
8       99       23     159.5  
9       71       11     162.5  
10      44       23     133.1  
11       1        3      64.1  
12      44       28     144.9

In [20]:
print('Bagel, Dave?')

Bagel, Dave?


In [28]:
from pymongo import MongoClient

In [29]:
# Connect to MongoDB
client =  MongoClient("mongodb://localhost:27017/")
db = client['2006_Draft_Analysis']
collection = db['2006_Draft_Analysis_Collection']

data_dict1 = df.to_dict("records")
data_dict2 = qb_data.to_dict("records")
data_dict3 = rb_data.to_dict("records")
data_dict4 = wr_te_data.to_dict("records")
data_dict5 = k_data.to_dict("records")
data_dict6 = p_data.to_dict("records")
data_dict7 = def_data.to_dict("records")


# # Insert collection
collection.insert_many(data_dict1)
collection.insert_many(data_dict2)
collection.insert_many(data_dict3)
collection.insert_many(data_dict4)
collection.insert_many(data_dict5)
collection.insert_many(data_dict6)
collection.insert_many(data_dict7)


In [30]:
df2 = pd.read_csv('Resources/2006_Draft_Class1.csv')
df2

name   first_name  last_name        birth_city  \
0           Mario Williams        Mario   Williams         Richlands   
1              Reggie Bush       Reggie       Bush     Spring Valley   
2              Vince Young        Vince      Young           Houston   
3    D'Brickashaw Ferguson  DBrickashaw   Ferguson          New York   
4                A.J. Hawk           AJ       Hawk       Centerville   
..                     ...          ...        ...               ...   
224            Ben Obomanu          Ben    Obomanu             Selma   
225         David Anderson        David   Anderson  Westlake Village   
226        Marques Colston      Marques    Colston        Harrisburg   
227         Dave Tollefson         Dave  Tollefson      Walnut Creek   
228                    NaN          NaN        NaN               NaN   

    birth_state birth_country  birth_date             college  \
0            NC           USA   1/31/1985  North Carolina St.   
1            CA           USA    3/2/1985                 USC   
2            TX           USA   5/18/1983               Texas   
3            NY           USA  12/10/1983            Virginia   
4            OH           USA    1/6/1984            Ohio St.   
..          ...           ...         ...                 ...   
224          AL           USA  10/30/1983              Auburn   
225          CA           USA   7/28/1983        Colorado St.   
226          PA           USA    6/5/1983             Hofstra   
227          CA           USA   5/19/1981     NW Missouri St.   
228         NaN           NaN         NaN                 NaN   

             draft_team draft_round  draft_pick  draft_year position  \
0        Houston Texans         1st         1.0      2006.0       DE   
1    New Orleans Saints         1st         2.0      2006.0       RB   
2      Tennessee Titans         1st         3.0      2006.0       QB   
3         New York Jets         1st         4.0      2006.0        T   
4     Green Bay Packers         1st         5.0      2006.0       LB   
..                  ...         ...         ...         ...      ...   
224    Seattle Seahawks         7th       249.0      2006.0       WR   
225      Houston Texans         7th       251.0      2006.0       WR   
226  New Orleans Saints         7th       252.0      2006.0       WR   
227   Green Bay Packers         7th       253.0      2006.0       DE   
228                 NaN         NaN         NaN         NaN      NaN   

     height_feet  height_inches height  
0            6.0            6.0    6-6  
1            6.0            0.0    6-0  
2            6.0            5.0    6-5  
3            6.0            5.0    6-5  
4            6.0            1.0    6-1  
..           ...            ...    ...  
224          6.0            0.0    6-0  
225          5.0           10.0   5-10  
226          6.0            4.0    6-4  
227          6.0            4.0    6-4  
228          NaN            NaN    NaN  

[229 rows x 16 columns]

In [31]:
client =  MongoClient("mongodb://localhost:27017/")
db = client['2006_Draft_Analysis']
collection = db['2006_Draft_Analysis_Collection']

data_dict8 = df2.to_dict("records")
collection.insert_many(data_dict8)